In [61]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [62]:
# loop through all states, or only generate the specified state
# state = 'ohio'
# state = 'florida'
# state = 'kansas'
# state = 'oregon'
# state = 'tennessee'
# state = 'illinois'
# state = 'washington'
# state = 'westvirginia'
state = 'wyoming'

In [63]:
#print current path
import os
print(os.getcwd())

/Users/lisawang/MIDS/opioid-2023-kml/10_Code/Transactions_DataProcessing


In [64]:
pop = pd.read_csv('../../00_Data/Population_Data/population.csv')
state_path = '../../00_Data/Transaction_Data/05_MME_Final/'+ state + '.parquet'
state_MME = pd.read_parquet(state_path)

In [65]:
per_capita_MME = state_MME.copy()
per_capita_MME['county_population'] = pd.NA
per_capita_MME['per_capita_MME_mg'] = pd.NA
for index, row in state_MME.iterrows():
    FIPS = row['FIPS']
    year = row['TransactionYear']
    target = pop[pop['FIPS'] == FIPS]
    if len(target==1):
        population = int(target[f'{year}'].values)
        #print(population)
        idx = per_capita_MME[(per_capita_MME['TransactionYear'] == year)&
                                        (per_capita_MME['FIPS'] == FIPS)].index
        per_capita_MME.loc[idx,'county_population'] = population
    else:
        print(f'Did not find data for FIPS: {FIPS}, year: {year}')

per_capita_MME['per_capita_MME_mg'] = per_capita_MME['morphine_equivalent_g'] / per_capita_MME['county_population'] * 1000
print(per_capita_MME.sample(5))
assert per_capita_MME[per_capita_MME['per_capita_MME_mg'].isna()].empty, 'There is missing per_capita_MME_mg'
assert per_capita_MME[per_capita_MME['county_population'].isna()].empty, 'There is missing county_population'

/var/folders/y6/lmkfsrf92b3dqv2xcg6777lh0000gn/T/ipykernel_18147/2672912089.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  population = int(target[f'{year}'].values)


    BUYER_STATE BUYER_COUNTY  TransactionYear  MME_Conversion_Factor  \
99           WY      JOHNSON             2015                  904.0   
127          WY      NATRONA             2013                11957.0   
139          WY     NIOBRARA             2015                  257.0   
24           WY     CAMPBELL             2010                 5559.5   
187          WY   SWEETWATER             2013                 7435.5   

     CALC_BASE_WT_IN_GM  morphine_equivalent_g   FIPS county_population  \
99          1516.774975            1954.603163  56019              8622   
127        25310.067842           32953.782183  56025             81175   
139          330.753005             383.086192  56027              2504   
24          9703.763175           12378.739875  56005             46253   
187        18566.892908           25476.868662  56037             45189   

    per_capita_MME_mg  
99         226.699509  
127        405.959744  
139        152.989693  
24          267.6310

In [66]:
# Step 4: Save the per capita dataframe back to a Parquet file
final_path = '../../00_Data/Transaction_Data/06_PerCapitaMME/' + state + '_PC.parquet'
per_capita_MME.to_parquet(final_path)